In [ ]:
!pip -q install faiss-cpu sentence-transformers transformers accelerate gradio gTTS

import torch
print('torch:', torch.__version__)
print('cuda available:', torch.cuda.is_available())

## 2) ایمپورت‌ها و تنظیمات

In [ ]:
import os
import re
import numpy as np
import faiss
from dataclasses import dataclass

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

from gtts import gTTS
import gradio as gr

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('DEVICE:', DEVICE)


In [ ]:
DOCUMENT_TEXT = """
""".strip()

def load_txt_from_path(path: str) -> str:
    with open(path, 'r', encoding='utf-8') as f:
        return f.read()


print('Document chars:', len(DOCUMENT_TEXT))


## 4) Chunking متن با overlap

در اینجا متن را به بخش‌های هم‌پوشان (Overlapping chunks) تبدیل می‌کنیم تا بازیابی دقیق‌تر شود.

In [ ]:
def normalize_text(text: str) -> str:
    text = text.replace('\u200c', ' ')  # ZWNJ
    text = re.sub(r"\s+", " ", text).strip()
    return text

def chunk_text(text: str, chunk_size: int = 450, overlap: int = 80):
    """Chunking ساده بر اساس تعداد کاراکتر
    chunk_size و overlap قابل تغییر هستند
    """
    text = normalize_text(text)
    if not text:
        return []
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = end - overlap
        if start < 0:
            start = 0
        if end == len(text):
            break
    return chunks

chunks = chunk_text(DOCUMENT_TEXT, chunk_size=450, overlap=80)
print('Num chunks:', len(chunks))
print('Sample chunk:\n', chunks[0][:300] if chunks else 'EMPTY')


## 5) ساخت embedding برای chunkها


In [ ]:
EMBED_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embedder = SentenceTransformer(EMBED_MODEL_NAME)

def embed_texts(texts):
    vecs = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True, normalize_embeddings=True)
    return vecs.astype('float32')

chunk_embeddings = embed_texts(chunks) if chunks else np.zeros((0, 384), dtype='float32')
print('Embeddings shape:', chunk_embeddings.shape)


In [ ]:
def build_faiss_index(embeddings: np.ndarray):
    if embeddings.size == 0:
        return None
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    return index

index = build_faiss_index(chunk_embeddings)
print('FAISS index ready:', index is not None)


## 7) بازیابی top-k بخش مرتبط با سوال

In [ ]:
def retrieve_top_k(query: str, k: int = 4):
    if index is None or not chunks:
        return []
    q_emb = embed_texts([query])
    scores, ids = index.search(q_emb, k)
    ids = ids[0].tolist()
    scores = scores[0].tolist()
    results = []
    for i, s in zip(ids, scores):
        if i == -1:
            continue
        results.append((chunks[i], float(s), i))
    return results

test_q = "موضوع سند چیست؟"
print(retrieve_top_k(test_q, k=3)[:1])


## 8) آماده‌سازی prompt (Context + Question + قوانین)

قانون اصلی:

- **Answer only from context**
- اگر پاسخ در متن نبود، دقیقاً بگو: «اطلاعات کافی در متن موجود نیست.»


In [ ]:
def build_prompt(context_chunks, question: str) -> str:
    context = "\n\n".join([f"[{i}] {c}" for i, c in enumerate(context_chunks, start=1)])
    prompt = f"""
You are a QA assistant.

RULES:
1) Answer ONLY using the provided CONTEXT.
2) If the answer is not in the context, say exactly: "اطلاعات کافی در متن موجود نیست."
3) Keep the answer concise and well-structured.

CONTEXT:
{context}

QUESTION:
{question}

ANSWER (in Persian):
""".strip()
    return prompt


## 9) تولید پاسخ با LLM رایگان

برای سازگاری چندزبانه و اجرای سبک، از `google/mt5-small` استفاده می‌کنیم.

> اگر GPU دارید، سرعت بهتر خواهد بود.

In [ ]:
LLM_NAME = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(LLM_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(LLM_NAME)
model.to(DEVICE)

def generate_answer(prompt: str, max_new_tokens: int = 180):
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True, max_length=1024).to(DEVICE)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            do_sample=False,
        )
    text = tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    return text

# تست سریع
if chunks:
    r = retrieve_top_k("سند درباره چیست؟", k=4)
    ctx = [x[0] for x in r]
    p = build_prompt(ctx, "سند درباره چیست؟")
    print(generate_answer(p))
else:
    print('Document is empty. Paste or upload a .txt first.')


## 10) تبدیل پاسخ به صوت (TTS)

از `gTTS` استفاده می‌کنیم (رایگان). خروجی فایل mp3 تولید می‌شود.

In [ ]:
from gtts import gTTS

def text_to_speech(text, out_path="answer.mp3", lang="en"):
    text = (text or "").strip()
    if not text:
        return None
    gTTS(text=text, lang=lang).save(out_path)
    return out_path



## 11) تابع کامل RAG (Retrieve → Prompt → Generate)

این تابع تضمین می‌کند که پاسخ فقط از context تولید شود. اگر سند خالی باشد هم پیام مناسب می‌دهد.

In [ ]:
def rag_answer(question: str, top_k: int = 4):
    if not DOCUMENT_TEXT.strip() or not chunks:
        return "ابتدا متن سند را وارد کنید (Paste یا فایل .txt).", []
    retrieved = retrieve_top_k(question, k=top_k)
    context_chunks = [c for (c, s, idx) in retrieved]
    prompt = build_prompt(context_chunks, question)
    answer = generate_answer(prompt)
    return answer, retrieved


## 12) رابط چت ساده با Gradio

In [ ]:
def rebuild_pipeline_with_new_doc(doc_text: str, chunk_size: int = 450, overlap: int = 80):
    global DOCUMENT_TEXT, chunks, chunk_embeddings, index
    DOCUMENT_TEXT = (doc_text or "").strip()
    chunks = chunk_text(DOCUMENT_TEXT, chunk_size=chunk_size, overlap=overlap)
    if chunks:
        chunk_embeddings = embed_texts(chunks)
        index = build_faiss_index(chunk_embeddings)
    else:
        chunk_embeddings = np.zeros((0, 384), dtype='float32')
        index = None
    return f"✅ سند بارگذاری شد. تعداد chunk: {len(chunks)}"

def read_uploaded_file(file_obj):
    if file_obj is None:
        return ""

    # حالت‌های رایج در Gradio
    if isinstance(file_obj, str):
        path = file_obj
    elif hasattr(file_obj, "name"):   # tempfile wrapper
        path = file_obj.name
    elif isinstance(file_obj, dict) and "path" in file_obj:
        path = file_obj["path"]
    elif hasattr(file_obj, "path"):
        path = file_obj.path
    else:
        raise TypeError(f"Unknown file_obj type: {type(file_obj)}")

    with open(path, "r", encoding="utf-8") as f:
        return f.read()


def chat_fn(message, history, top_k, chunk_size, overlap):
    if not (DOCUMENT_TEXT and DOCUMENT_TEXT.strip()):
        # سه خروجی مطابق outputs
        return (history or []), "❌ ابتدا سند را وارد کنید.", None

    answer, retrieved = rag_answer(message, top_k=int(top_k))
    answer = (answer or "").strip()

    if not answer:
        answer = "متأسفانه پاسخ قابل تولید نیست. لطفاً سوال را واضح‌تر بپرسید یا سند را دوباره بارگذاری کنید."

    audio_path = text_to_speech(answer, out_path="answer.mp3", lang="en")


    sources_md = "\n\n".join([
        f"**Chunk #{idx} | score={score:.3f}**\n\n{chunk[:700]}"
        for (chunk, score, idx) in retrieved
    ])
    history = (history or []) + [(message, answer)]
    return history, sources_md, audio_path


with gr.Blocks() as demo:
    gr.Markdown("# RAG QA + TTS (سندمحور)")
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## 1) ورود سند")
            doc_paste = gr.Textbox(label="متن سند (Paste)", lines=10)
            doc_file = gr.File(label="یا آپلود فایل .txt", file_types=['.txt'])
            chunk_size = gr.Slider(200, 1200, value=450, step=10, label="Chunk size (chars)")
            overlap = gr.Slider(0, 300, value=80, step=10, label="Overlap (chars)")
            load_btn = gr.Button("بارگذاری/بازسازی ایندکس")
            load_status = gr.Textbox(label="وضعیت", interactive=False)

        with gr.Column(scale=2):
            gr.Markdown("## 2) چت")
            chatbot = gr.Chatbot(label="گفتگو")
            msg = gr.Textbox(label="سوال خود را بنویسید و Enter بزنید")
            top_k = gr.Slider(1, 8, value=4, step=1, label="Top-k retrieval")
            sources = gr.Markdown(label="بخش‌های بازیابی‌شده")
            audio = gr.Audio(label="صدای پاسخ", type="filepath")

    def on_load(doc_text, file_obj, chunk_size, overlap):
    # اولویت: اگر paste پر بود همان را بگیر
      if doc_text and doc_text.strip():
          chosen_text = doc_text
      # وگرنه از فایل بخوان
      elif file_obj is not None:
          chosen_text = read_uploaded_file(file_obj)
      else:
          chosen_text = ""

      if not (chosen_text and chosen_text.strip()):
          return "❌ سند خالی است. متن را Paste کنید یا فایل .txt آپلود کنید."

      return rebuild_pipeline_with_new_doc(chosen_text, int(chunk_size), int(overlap))


    load_btn.click(on_load, inputs=[doc_paste, doc_file, chunk_size, overlap], outputs=[load_status])
    # وقتی paste تغییر کرد => ایندکس بساز
    doc_paste.change(
        on_load,
        inputs=[doc_paste, doc_file, chunk_size, overlap],
        outputs=[load_status]
    )

    # وقتی فایل آپلود شد => ایندکس بساز
    doc_file.change(
        on_load,
        inputs=[doc_paste, doc_file, chunk_size, overlap],
        outputs=[load_status]
    )
    chunk_size.change(
        on_load,
        inputs=[doc_paste, doc_file, chunk_size, overlap],
        outputs=[load_status]
    )

    overlap.change(
        on_load,
        inputs=[doc_paste, doc_file, chunk_size, overlap],
        outputs=[load_status]
    )


    msg.submit(chat_fn, inputs=[msg, chatbot, top_k, chunk_size, overlap], outputs=[chatbot, sources, audio])
    msg.submit(lambda: "", None, msg)

demo.launch(share=True, debug=True)
